## This notebook contains code for training MLP - for combining scores

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('submission_train.csv')
samuels_pred_df = pd.read_csv('training_no_duplicates.csv')
num_rows =1127644
# Split the DataFrame into two parts
first_part = train_df.iloc[:num_rows]
second_part = train_df.iloc[num_rows:]
# Concatenate the second part first, followed by the first part
train_df = pd.concat([second_part, first_part]).reset_index(drop=True)
train_df.rename(columns={'sentiment_scores':'sentiment_scores2'}, inplace=True)
train_df.loc[:, 'sentiment_scores'] = samuels_pred_df['predictions']

#train_df.loc[:, 'sentiment_scores'] = train_df.loc[:, 'sentiment_scores'].replace(0,-1)

# For these features, 0 means non existent
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].replace(0, np.nan)
train_df.loc[:,'spam_score'] = train_df['spam_score'].replace(0,np.nan)
train_df.loc[:,'fake_lables'] = train_df['fake_lables'].replace(0,np.nan)


def modify_value(x):
    if x == 0.0 or np.isnan(x):
        return np.nan
    elif x > 0.0:
        return 1
    else:
        return -1

train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].apply(modify_value)
train_df.loc[:, 'vadar'] = train_df['vadar'].apply(modify_value)

In [ ]:
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].replace(-1, 0)
train_df.loc[:,'spam_score'] = train_df['spam_score'].replace(1, 0)
train_df.loc[:,'fake_lables'] = train_df['fake_lables'].replace(-1,0)
train_df.loc[:,'sentiment_scores2'] = train_df['sentiment_scores2'].replace(-1,0)
train_df.loc[:,'emotion_scores'] = train_df['emotion_scores'].replace(-1,0)
train_df.loc[:,'label'] = train_df['emotion_scores'].replace(-1,0)


In [ ]:
train_df.tail()


,line,label,emoji_scores,fake_lables,spam_score,vadar,emotion_scores,sentiment_scores2,sentiment_scores
2270477,a warning sign ? rt <user> the negativity you ...,1,0.0,0.0,NaN,-1.0,1,0,0
2270478,<user> ff too thank youuu,1,1.0,1.0,NaN,1.0,1,1,1
2270479,i just love shumpa ! that s my girl,1,NaN,NaN,NaN,1.0,1,1,1
2270480,the best way to start a day ! no matter what h...,1,NaN,NaN,NaN,1.0,1,1,1
2270481,frenchies want dtou i am not from french but <...,1,NaN,NaN,NaN,1.0,1,1,1


In [ ]:

## Nulling out the emotion_scores
train_df.loc[train_df['emoji_scores'] == 0, 'emotion_scores'] = np.nan
#train_df.loc[train_df['emoji_scores'] == 0, 'vadar'] = np.nan
train_df.loc[train_df['fake_lables'] == 0, 'emotion_scores'] = np.nan
#train_df.loc[train_df['fake_lables'] == 0, 'vadar'] = np.nan
train_df.loc[train_df['spam_score'] == 0, 'emotion_scores'] = np.nan
#train_df.loc[train_df['spam_score'] == 0, 'vadar'] = np.nan

In [ ]:

train_df['spam_score'].fillna(train_df['sentiment_scores'], inplace=True)
train_df['fake_lables'].fillna(train_df['sentiment_scores'], inplace=True)
train_df['emoji_scores'].fillna(train_df['sentiment_scores'], inplace=True)
#train_df['vadar'].fillna(train_df['sentiment_scores'], inplace=True)
train_df['emotion_scores'].fillna(train_df['sentiment_scores'], inplace=True)


## Make sure all are class 0 instead of -1

In [ ]:

train_df.drop(columns=['vadar'], inplace=True)

In [ ]:
rows_with_minus_one = train_df.apply(lambda row: -1 in row.values, axis=1)


In [ ]:
print("Rows that contain -1:")
print(train_df[rows_with_minus_one])

Rows that contain -1:
Empty DataFrame
Columns: [line, label, emoji_scores, fake_lables, spam_score, emotion_scores, sentiment_scores2, sentiment_scores]
Index: []


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Example: Load your dataset
# X, y = load_your_data()  # Replace with your data loading method
# For demonstration, let's create a synthetic dataset
from sklearn.datasets import make_classification
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables','emotion_scores','sentiment_scores']]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Standardize the data (mean=0, variance=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=42,initial_strategy='most_frequent')

# Fit the imputer on the training set and transform the training data
X_train = imputer.fit_transform(X_train)

# Transform the testing data using the same imputer
X_test = imputer.transform(X_test)

# Convert to NumPy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=4,pin_memory =True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=4,pin_memory =True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1816385, 5)
(454097, 5)
(1816385,)
(454097,)


In [ ]:
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # 64 doubled to 128
        self.fc2 = nn.Linear(128, 64)         # 32 doubled to 64
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [ ]:

# Initialize the model
model = MLP(X_train.shape[1])

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Move model to device
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')


Using device: cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1/10, Loss: 0.25915450665406026
Epoch 2/10, Loss: 0.2585157973362463
Epoch 3/10, Loss: 0.25842708534571973
Epoch 4/10, Loss: 0.2583674266579999
Epoch 5/10, Loss: 0.25834742009362394
Epoch 6/10, Loss: 0.258318503519695
Epoch 7/10, Loss: 0.25829402006028634
Epoch 8/10, Loss: 0.2583063428112482
Epoch 9/10, Loss: 0.25828319187127335
Epoch 10/10, Loss: 0.2582861891330825


In [ ]:
# Evaluation
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        predictions = (outputs > 0.5).float()
        y_pred.extend(predictions.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())


# Convert to numpy arrays
y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Print classification report
# Assuming y_true and y_pred are already defined
report = classification_report(y_true, y_pred, output_dict=True)

# Convert the report to a DataFrame
report_df = pd.DataFrame(report).transpose()

# Format the values to 4 decimal places
report_df = report_df.applymap(lambda x: f"{x:.4f}" if isinstance(x, float) else x)

# Print the formatted classification report
print(report_df)

             precision  recall f1-score      support
0.0             0.7495  0.9951   0.8550  160771.0000
1.0             0.9967  0.8177   0.8984  293326.0000
accuracy        0.8805  0.8805   0.8805       0.8805
macro avg       0.8731  0.9064   0.8767  454097.0000
weighted avg    0.9092  0.8805   0.8830  454097.0000


In [ ]:
torch.save(model.state_dict(), 'mlp_submission3.pth')